# Temporal Difference: On-policy Double Sarsa, Stochastic

In [1]:
import numpy as np

## Create environment

In [2]:
def create_environment_states():
    """Creates environment states.

    Returns:
        num_states: int, number of states.
        num_terminal_states: int, number of terminal states.
        num_non_terminal_states: int, number of non terminal states.
    """
    num_states = 16
    num_terminal_states = 2
    num_non_terminal_states = num_states - num_terminal_states

    return num_states, num_terminal_states, num_non_terminal_states

In [3]:
def create_environment_actions(num_non_terminal_states):
    """Creates environment actions.

    Args:
        num_non_terminal_states: int, number of non terminal states.

    Returns:
        max_num_actions: int, max number of actions possible.
        num_actions_per_non_terminal_state: array[int], number of actions per
            non terminal state.
    """
    max_num_actions = 4

    num_actions_per_non_terminal_state = np.repeat(
        a=max_num_actions, repeats=num_non_terminal_states)

    return max_num_actions, num_actions_per_non_terminal_state

In [4]:
def create_environment_successor_counts(num_states, max_num_actions):
    """Creates environment successor counts.

    Args:
        num_states: int, number of states.
        max_num_actions: int, max number of actions possible.
    Returns:
        num_state_action_successor_states: array[int], number of successor
            states s' that can be reached from state s by taking action a.
    """
    num_state_action_successor_states = np.repeat(
        a=1, repeats=num_states * max_num_actions)

    num_state_action_successor_states = np.reshape(
        a=num_state_action_successor_states,
        newshape=(num_states, max_num_actions))

    return num_state_action_successor_states

In [5]:
def create_environment_successor_arrays(
        num_non_terminal_states, max_num_actions):
    """Creates environment successor arrays.

    Args:
        num_non_terminal_states: int, number of non terminal states.
        max_num_actions: int, max number of actions possible.
    Returns:
        sp_idx: array[int], state indices of new state s' of taking action a
            from state s.
        p: array[float], transition probability to go from state s to s' by
            taking action a.
        r: array[float], reward from new state s' from state s by taking
            action a.
    """
    sp_idx = np.array(
        object=[1, 0, 14, 4,
                2, 1, 0, 5,
                2, 2, 1, 6,
                4, 14, 3, 7,
                5, 0, 3, 8,
                6, 1, 4, 9,
                6, 2, 5, 10,
                8, 3, 7, 11,
                9, 4, 7, 12,
                10, 5, 8, 13,
                10, 6, 9, 15,
                12, 7, 11, 11,
                13, 8, 11, 12,
                15, 9, 12, 13],
        dtype=np.int64)

    p = np.repeat(
        a=1.0, repeats=num_non_terminal_states * max_num_actions * 1)

    r = np.repeat(
        a=-1.0, repeats=num_non_terminal_states * max_num_actions * 1)

    sp_idx = np.reshape(
        a=sp_idx,
        newshape=(num_non_terminal_states, max_num_actions, 1))
    p = np.reshape(
        a=p,
        newshape=(num_non_terminal_states, max_num_actions, 1))
    r = np.reshape(
        a=r,
        newshape=(num_non_terminal_states, max_num_actions, 1))

    return sp_idx, p, r

In [6]:
def create_environment():
    """Creates environment.

    Returns:
        num_states: int, number of states.
        num_terminal_states: int, number of terminal states.
        num_non_terminal_states: int, number of non terminal states.
        max_num_actions: int, max number of actions possible.
        num_actions_per_non_terminal_state: array[int], number of actions per
            non terminal state.
        num_state_action_successor_states: array[int], number of successor
            states s' that can be reached from state s by taking action a.
        sp_idx: array[int], state indices of new state s' of taking action a
            from state s.
        p: array[float], transition probability to go from state s to s' by
            taking action a.
        r: array[float], reward from new state s' from state s by taking
            action a.
    """
    (num_states,
     num_terminal_states,
     num_non_terminal_states) = create_environment_states()

    (max_num_actions,
     num_actions_per_non_terminal_state) = create_environment_actions(
        num_non_terminal_states)

    num_state_action_successor_states = create_environment_successor_counts(
        num_states, max_num_actions)

    (sp_idx,
     p,
     r) = create_environment_successor_arrays(
        num_non_terminal_states, max_num_actions)

    return (num_states,
            num_terminal_states,
            num_non_terminal_states,
            max_num_actions,
            num_actions_per_non_terminal_state,
            num_state_action_successor_states,
            sp_idx,
            p,
            r)

## Set hyperparameters

In [7]:
def set_hyperparameters():
    """Sets hyperparameters.

    Returns:
        num_episodes: int, number of episodes to train over.
        maximum_episode_length: int, max number of timesteps for an episode.
        alpha: float, alpha > 0, learning rate.
        epsilon: float, 0 <= epsilon <= 1, exploitation-exploration trade-off,
            higher means more exploration.
        gamma: float, 0 <= gamma <= 1, amount to discount future reward.
    """
    num_episodes = 10000
    maximum_episode_length = 200
    alpha = 0.1
    epsilon = 0.1
    gamma = 1.0

    return num_episodes, maximum_episode_length, alpha, epsilon, gamma

## Create value function and policy arrays

In [8]:
def create_value_function_arrays(num_states, max_num_actions):
    """Creates value function arrays.

    Args:
        num_states: int, number of states.
        max_num_actions: int, max number of actions possible.
    Returns:
        q1: array[float], keeps track of the estimated value of each
            state-action pair Q1(s, a).
        q2: array[float], keeps track of the estimated value of each
            state-action pair Q2(s, a).
    """
    q1 = np.repeat(a=0.0, repeats=num_states * max_num_actions)
    q1 = np.reshape(a=q1, newshape=(num_states, max_num_actions))

    q2 = np.repeat(a=0.0, repeats=num_states * max_num_actions)
    q2 = np.reshape(a=q2, newshape=(num_states, max_num_actions))

    return q1, q2

In [9]:
def create_policy_arrays(num_non_terminal_states, max_num_actions):
    """Creates policy arrays.

    Args:
        num_non_terminal_states: int, number of non terminal states.
        max_num_actions: int, max number of actions possible.
    Returns:
        policy: array[float], learned stochastic policy of which
            action a to take in state s.
    """
    policy = np.repeat(
        a=1.0 / max_num_actions,
        repeats=num_non_terminal_states * max_num_actions)

    policy = np.reshape(
        a=policy,
        newshape=(num_non_terminal_states, max_num_actions))

    return policy

## Create algorithm

In [10]:
# Set random seed so that everything is reproducible
np.random.seed(seed=0)

In [11]:
def initialize_epsiode(
        num_non_terminal_states, max_num_actions, q1, q2, epsilon, policy):
    """Initializes epsiode with initial state, initial action, and policy.

    Args:
        num_non_terminal_states: int, number of non terminal states.
        max_num_actions: int, max number of actions possible.
        q1: array[float], keeps track of the estimated value of each
            state-action pair Q1(s, a).
        q2: array[float], keeps track of the estimated value of each
            state-action pair Q2(s, a).
        epsilon: float, 0 <= epsilon <= 1, exploitation-exploration trade-off,
            higher means more exploration.
        policy: array[float], learned stochastic policy of which
            action a to take in state s.
    Returns:
        init_s_idx: int, initial state index from set of non terminal states.
        init_a_idx: int, initial action index from set of actions of state
            init_s_idx.
        policy: array[float], learned stochastic policy of which
            action a to take in state s.
    """
    # Randomly choose an initial state from all non-terminal states
    init_s_idx = np.random.randint(
        low=0, high=num_non_terminal_states, dtype=np.int64)

    # Choose policy for chosen state by epsilon-greedy choosing from the
    # state-action-value function
    policy = epsilon_greedy_policy_from_state_action_function(
        max_num_actions, q1, q2, epsilon, init_s_idx, policy)

    # Get initial action
    init_a_idx = np.random.choice(
        a=max_num_actions, p=policy[init_s_idx, :])

    return init_s_idx, init_a_idx, policy

In [12]:
def epsilon_greedy_policy_from_state_action_function(
        max_num_actions, q1, q2, epsilon, s_idx, policy):
    """Create epsilon-greedy policy from state-action value function.

    Args:
        max_num_actions: int, max number of actions possible.
        q1: array[float], keeps track of the estimated value of each
            state-action pair Q1(s, a).
        q2: array[float], keeps track of the estimated value of each
            state-action pair Q2(s, a).
        epsilon: float, 0 <= epsilon <= 1, exploitation-exploration trade-off,
            higher means more exploration.
        s_idx: int, current state index.
        policy: array[float], learned stochastic policy of which action a to
            take in state s.
    Returns:
        policy: array[float], learned stochastic policy of which action a to
            take in state s.
    """
    # Combine state-action value functions
    q = q1[s_idx, :] + q2[s_idx, :]

    # Save max state-action value and find the number of actions that have the
    # same max state-action value
    max_action_value = np.max(a=q)
    max_action_count = np.count_nonzero(a=q == max_action_value)

    # Apportion policy probability across ties equally for state-action pairs
    # that have the same value and zero otherwise
    if max_action_count == max_num_actions:
        max_policy_prob_per_action = 1.0 / max_action_count
        remain_prob_per_action = 0.0
    else:
        max_policy_prob_per_action = (1.0 - epsilon) / max_action_count
        remain_prob_per_action = epsilon / (max_num_actions - max_action_count)

    policy[s_idx, :] = np.where(
        q == max_action_value,
        max_policy_prob_per_action,
        remain_prob_per_action)

    return policy

In [13]:
def loop_through_episode(
        num_non_terminal_states,
        max_num_actions,
        num_state_action_successor_states,
        sp_idx,
        p,
        r,
        q1,
        q2,
        policy,
        alpha,
        epsilon,
        gamma,
        maximum_episode_length,
        s_idx,
        a_idx):
    """Loops through episode to iteratively update policy.

    Args:
        num_non_terminal_states: int, number of non terminal states.
        max_num_actions: int, max number of actions possible.
        num_state_action_successor_states: array[int], number of successor
            states s' that can be reached from state s by taking action a.
        sp_idx: array[int], state indices of new state s' of taking action a
            from state s.
        p: array[float], transition probability to go from state s to s' by
            taking action a.
        r: array[float], reward from new state s' from state s by taking
            action a.
        q1: array[float], keeps track of the estimated value of each
            state-action pair Q1(s, a).
        q2: array[float], keeps track of the estimated value of each
            state-action pair Q2(s, a).
        policy: array[float], learned stochastic policy of which
            action a to take in state s.
        alpha: float, alpha > 0, learning rate.
        epsilon: float, 0 <= epsilon <= 1, exploitation-exploration trade-off,
            higher means more exploration.
        gamma: float, 0 <= gamma <= 1, amount to discount future reward.
        maximum_episode_length: int, max number of timesteps for an episode.
        s_idx: int, current state index.
        a_idx: int, current action index.
    Returns:
        q1: array[float], keeps track of the estimated value of each
            state-action pair Q1(s, a).
        q2: array[float], keeps track of the estimated value of each
            state-action pair Q2(s, a).
        policy: array[float], learned stochastic policy of which
            action a to take in state s.
    """
    # Loop through episode steps until termination
    for t in range(0, maximum_episode_length):
        # Get reward
        successor_state_transition_idx = np.random.choice(
            a=num_state_action_successor_states[s_idx, a_idx],
            p=p[s_idx, a_idx, :])

        reward = r[s_idx, a_idx, successor_state_transition_idx]

        # Get next state
        next_s_idx = sp_idx[s_idx, a_idx, successor_state_transition_idx]

        # Update state action value equally randomly selecting from the
        # state-action-value functions
        if (np.random.randint(low=0, high=2, dtype=np.int64) == 0):
            q1, q2, policy, s_idx, a_idx = update_q(
                num_non_terminal_states,
                max_num_actions,
                q2,
                policy,
                alpha,
                epsilon,
                gamma,
                s_idx,
                a_idx,
                reward,
                next_s_idx,
                q1)
        else:
            q2, q1, policy, s_idx, a_idx = update_q(
                num_non_terminal_states,
                max_num_actions,
                q1,
                policy,
                alpha,
                epsilon,
                gamma,
                s_idx,
                a_idx,
                reward,
                next_s_idx,
                q2)

        if next_s_idx >= num_non_terminal_states:
            break  # episode terminated since we ended up in a terminal state

    return q1, q2, policy

In [14]:
# This function updates the state-action-value function
def update_q(
        num_non_terminal_states,
        max_num_actions,
        not_updating_q,
        policy,
        alpha,
        epsilon,
        gamma,
        s_idx,
        a_idx,
        reward,
        next_s_idx,
        updating_q):
    """Updates state-action-value function using multiple estimates.

    Args:
        num_non_terminal_states: int, number of non terminal states.
        max_num_actions: int, max number of actions possible.
        not_updating_q: array[float], keeps track of the estimated value of each
            state-action pair Q(s, a) that is not updated this iteration.
        policy: array[float], learned stochastic policy of which
            action a to take in state s.
        alpha: float, alpha > 0, learning rate.
        epsilon: float, 0 <= epsilon <= 1, exploitation-exploration trade-off,
            higher means more exploration.
        gamma: float, 0 <= gamma <= 1, amount to discount future reward.
        s_idx: int, current state index.
        a_idx: int, current action index.
        reward: float, current reward from taking action a_idx in state s_idx.
        next_s_idx: int, next state index.
        updating_q: array[float], keeps track of the estimated value of each
            state-action pair Q(s, a) that is updated this iteration.
    Returns:
        updating_q: array[float], keeps track of the estimated value of each
            state-action pair Q(s, a) that is updated this iteration.
        not_updating_q: array[float], keeps track of the estimated value of each
            state-action pair Q(s, a) that is not updated this iteration.
        policy: array[float], learned stochastic policy of which
            action a to take in state s.
        s_idx: int, new current state index.
        a_idx: int, new current action index.
    """
    # Check to see if we actioned into a terminal state
    if next_s_idx >= num_non_terminal_states:
        updating_q[s_idx, a_idx] += alpha * (reward - updating_q[s_idx, a_idx])
    else:
        # Choose policy for chosen state by epsilon-greedy choosing from the
        # state-action-value function
        policy = epsilon_greedy_policy_from_state_action_function(
            max_num_actions,
            updating_q,
            not_updating_q,
            epsilon,
            next_s_idx,
            policy)

        # Get next action
        next_a_idx = np.random.choice(
            a=max_num_actions, p=policy[next_s_idx, :])

        # Calculate state-action-function using quintuple SARSA
        delta = gamma * not_updating_q[next_s_idx, next_a_idx]
        delta -= updating_q[s_idx, a_idx]
        updating_q[s_idx, a_idx] += alpha * (reward + delta)

        # Update state and action to next state and action
        s_idx = next_s_idx
        a_idx = next_a_idx

    return updating_q, not_updating_q, policy, s_idx, a_idx

In [15]:
def on_policy_temporal_difference_double_sarsa(
        num_non_terminal_states,
        max_num_actions,
        num_state_action_successor_states,
        sp_idx,
        p,
        r,
        q1,
        q2,
        policy,
        alpha,
        epsilon,
        gamma,
        maximum_episode_length,
        num_episodes):
    """Loops through episodes to iteratively update policy.

    Args:
        num_non_terminal_states: int, number of non terminal states.
        max_num_actions: int, max number of actions possible.
        num_state_action_successor_states: array[int], number of successor
            states s' that can be reached from state s by taking action a.
        sp_idx: array[int], state indices of new state s' of taking action a
            from state s.
        p: array[float], transition probability to go from state s to s' by
            taking action a.
        r: array[float], reward from new state s' from state s by taking
            action a.
        q1: array[float], keeps track of the estimated value of each
            state-action pair Q1(s, a).
        q2: array[float], keeps track of the estimated value of each
            state-action pair Q2(s, a).
        policy: array[float], learned stochastic policy of which
            action a to take in state s.
        alpha: float, alpha > 0, learning rate.
        epsilon: float, 0 <= epsilon <= 1, exploitation-exploration trade-off,
            higher means more exploration.
        gamma: float, 0 <= gamma <= 1, amount to discount future reward.
        maximum_episode_length: int, max number of timesteps for an episode.
        num_episodes: int, number of episodes to train over.
    Returns:
        q1: array[float], keeps track of the estimated value of each
            state-action pair Q1(s, a).
        q2: array[float], keeps track of the estimated value of each
            state-action pair Q2(s, a).
        policy: array[float], learned stochastic policy of which
            action a to take in state s.
    """
    for episode in range(0, num_episodes):
        # Initialize episode to get initial state and action
        init_s_idx, init_a_idx, policy = initialize_epsiode(
            num_non_terminal_states, max_num_actions, q1, q2, epsilon, policy)

        # Loop through episode and update the policy
        q1, q2, policy = loop_through_episode(
            num_non_terminal_states,
            max_num_actions,
            num_state_action_successor_states,
            sp_idx,
            p,
            r,
            q1,
            q2,
            policy,
            alpha,
            epsilon,
            gamma,
            maximum_episode_length,
            init_s_idx,
            init_a_idx)

    return q1, q2, policy

## Run algorithm

In [16]:
def run_algorithm():
    """Runs the algorithm."""
    (num_states,
     num_terminal_states,
     num_non_terminal_states,
     max_num_actions,
     num_actions_per_non_terminal_state,
     num_state_action_successor_states,
     sp_idx,
     p,
     r) = create_environment()

    (num_episodes,
     maximum_episode_length,
     alpha,
     epsilon,
     gamma) = set_hyperparameters()

    q1, q2 = create_value_function_arrays(num_states, max_num_actions)

    policy = create_policy_arrays(num_non_terminal_states, max_num_actions)

    # Print initial arrays
    print("\nInitial state-action value function1")
    print(q1)

    print("\nInitial state-action value function2")
    print(q2)

    print("\nInitial policy")
    print(policy)

    # Run on policy temporal difference double sarsa
    q1, q2, policy = on_policy_temporal_difference_double_sarsa(
        num_non_terminal_states,
        max_num_actions,
        num_state_action_successor_states,
        sp_idx,
        p,
        r,
        q1,
        q2,
        policy,
        alpha,
        epsilon,
        gamma,
        maximum_episode_length,
        num_episodes)

    # Print final results
    print("\nFinal state-action value function1")
    print(q1)

    print("\nFinal state-action value function2")
    print(q2)

    print("\nFinal policy")
    print(policy)

In [17]:
run_algorithm()


Initial state-action value function1
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]

Initial state-action value function2
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]

Initial policy
[[0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]]

Final state-action value function1
[[-3.23820201 -2.10678805 -1.         -3.28034689]
 [-4.10314701 -3